# Automated ML

### Create workspace and experiment instances

In [1]:
from azureml.core import Workspace, Experiment
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-spam-classification-experiment'

experiment=Experiment(ws, experiment_name)

###  Dataset Overview
The dataset we are using is a spam classification dataset obtained from kaggle. We are going to perform multi-class text classification. 

### Get data

In [23]:
from azureml.core import Dataset

try: 
    training_dataset = Dataset.get_by_name(ws, name='capstone-spam-dataset')

except: # data is not found in the workspace
    print("Data not found in workspace. Please upload and register the data!")

### View the dataset

In [24]:
training_dataset.to_pandas_dataframe()

,Category,Message,Column3
0,ham,"Go until jurong point, crazy.. Available only ...",None
1,ham,Ok lar... Joking wif u oni...,None
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,None
3,ham,U dun say so early hor... U c already then say...,None
4,ham,"Nah I don't think he goes to usf, he lives aro...",None
...,...,...,...
5569,spam,This is the 2nd time we have tried 2 contact u...,None
5570,ham,Will ü b going to esplanade fr home?,None
5571,ham,"Pity, * was in mood for that. So...any other s...",None
5572,ham,The guy did some bitching but I acted like i'd...,None


### Connect to compute target

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
cluster_name = "capstone-compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


### AutoML Configuration

- task: "classification" because we are performing multi-class classification
- label_column_name: "Category" because we want to predict that 
- experiment_timeout_hours: 0.25 because we want the experiment to run for max 15 min

In [5]:
from azureml.train.automl import AutoMLConfig

automl_settings = {
    "n_cross_validations": 2,
    "primary_metric": 'accuracy',
    "enable_early_stopping": True,
    "max_concurrent_iterations": 5,
    "experiment_timeout_hours": 0.25,
    "featurization": 'auto',
}

automl_config = AutoMLConfig(
    task = 'classification',
    compute_target = compute_target,
    training_data = training_dataset,
    label_column_name = 'Category',
    **automl_settings
)

### Submit the experiment


In [6]:
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
capstone-spam-classification-experiment,AutoML_264ce6fc-7280-41c7-9d44-b02c767bf5bd,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


### Run Details

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

In [8]:
remote_run.wait_for_completion()

### Get the best model and display its properties

In [ ]:
best_run, fitted_model = remote_run.get_output()

# Get best_run metrics
print(best_run)
best_run_metrics = best_run.get_metrics()
for name, value in best_run_metrics.items():
    print(f"{name}: {value}")

In [ ]:
best_run.get_file_names()

### Best parameters of AutoML model

In [42]:
fitted_model.steps[1][1].get_params()

{'base_learners': None,
 'meta_learner': None,
 'training_cv_folds': None,
 '15': Pipeline(memory=None,
          steps=[('standardscalerwrapper',
                  StandardScalerWrapper(copy=True, with_mean=False, with_std=False)),
                 ('logisticregression',
                  LogisticRegression(C=51.79474679231202, class_weight=None,
                                     dual=False, fit_intercept=True,
                                     intercept_scaling=1, l1_ratio=None,
                                     max_iter=100, multi_class='ovr', n_jobs=1,
                                     penalty='l2', random_state=None,
                                     solver='lbfgs', tol=0.0001, verbose=0,
                                     warm_start=False))],
          verbose=False),
 '36': Pipeline(memory=None,
          steps=[('standardscalerwrapper',
                  StandardScalerWrapper(copy=True, with_mean=False, with_std=False)),
                 ('logisticregression',


### Save the model

In [ ]:
import joblib
joblib.dump(fitted_model, 'best-automl-model.pkl')

### Register the best model

In [ ]:
from azureml.core import Model
model = Model.register(
    workspace=ws, 
    model_name='best-automl-model', 
    model_path='./best-automl-model.pkl'
)

# Model Deployment

### Create an inference config

In [ ]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig

# Get the environment
from azureml.automl.core.shared import constants

best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'conda_dependencies.yml')
env = Environment.from_conda_specification(name='deployment-env', file_path='conda_dependencies.yml')

inference_config = InferenceConfig(
    environment=env,
    source_directory=".",
    entry_script="./automl_score.py",
)

### Define the deployment config - we deploy on Azure Container Instance (ACI)

In [ ]:
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=1, memory_gb=2, auth_enabled=True, enable_app_insights=True
)

### Deploy the model as a web service

In [ ]:
from azureml.core.model import Model
service = Model.deploy(
    ws,
    "automl-service",
    [model],
    inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

### Send a request to the web service you deployed to test it

In [25]:
import requests
import json
from azureml.core import Webservice

service = Webservice(workspace=ws, name="automl-service")
scoring_uri = service.scoring_uri

# If the service is authenticated, set the key or token
key, _ = service.get_keys()

# Set the appropriate headers
headers = {"Content-Type": "application/json"}
headers["Authorization"] = f"Bearer {key}"

# Make the request and display the response and logs
data =  {
  "Inputs": {
    "data": [
      {
        "Message": "Welcome to Airways!",
        "Column3": ""
     }
    ]
  }
}

input_data = json.dumps(data)
resp = requests.post(scoring_uri, data=input_data, headers=headers)
print(resp.json())

{"result": ["ham"]}


### Print the logs of the web service

In [22]:
print(service.get_logs())

/bin/bash: /azureml-envs/azureml_944df6c9e2b12a3bdcde13b5b8baccf0/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_944df6c9e2b12a3bdcde13b5b8baccf0/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_944df6c9e2b12a3bdcde13b5b8baccf0/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_944df6c9e2b12a3bdcde13b5b8baccf0/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2022-09-26T07:26:28,453535500+00:00 - iot-server/run 
2022-09-26T07:26:28,460346000+00:00 - rsyslog/run 
bash: /azureml-envs/azureml_944df6c9e2b12a3bdcde13b5b8baccf0/lib/libtinfo.so.6: no version information available (required by bash)
2022-09-26T07:26:28,488064600+00:00 - gunicorn/run 
2022-09-26T07:26:28,491386500+00:00 - nginx/run 
2022-09-26T07:26:28,506808600+00:00 | gunicorn/run | 
2022-09-26T07:26:28,521812500+00:00 | gu

### Delete the web service

In [23]:
service.delete()

### Shutdown the computes

In [26]:
try:
    instance = ComputeTarget(workspace=ws, name=cluster_name)

    instance.delete()
    instance.wait_for_completion(show_output=True)
    print('Deleted compute resource')

except ComputeTargetException as e:
    print('Already deleted!')

InProgress.....Current provisioning state of AmlCompute is "Deleting"

..
SucceededProvisioning operation finished, operation "Succeeded"
ComputeTargetException:
	Message: ComputeTargetNotFound: Compute Target with name capstone-compute-cluster not found in provided workspace
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "ComputeTargetNotFound: Compute Target with name capstone-compute-cluster not found in provided workspace"
    }
}
Already deleted!


**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
